In [300]:
import pandas as pd
import numpy as np
import datetime
from datetime import timedelta
from time import strftime
import plotly.express as px
import glob
import os
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler


In [301]:
# Missing Data from November 8th, 2020 to January 11st, 2021
files = os.path.join("C:\\Users\\aaliy\\OneDrive\\Documents\\GitHub\\energyResearch\\", "bingData*.csv")
files = glob.glob(files)

bingData = pd.concat(map(pd.read_csv, files))
bingData

,START_TIME,END_TIME,BUILDING_ID,TYPE,VALUE
0,2019-01-01 00:10:07,2019-01-01 01:10:07,MH,E,8.669
1,2019-01-01 00:10:07,2019-01-01 01:10:07,LA,E,6.808
2,2019-01-01 00:10:07,2019-01-01 01:10:07,KH,E,6.475
3,2019-01-01 00:10:07,2019-01-01 01:10:07,JH,E,8.623
4,2019-01-01 00:10:07,2019-01-01 01:10:07,HE,E,5.472
...,...,...,...,...,...
115590,2021-12-31 22:10:09,2021-12-31 23:10:09,HSB,E,36.000
115591,2021-12-31 22:10:09,2021-12-31 23:10:09,HSB,H,NaN
115592,2021-12-31 22:10:09,2021-12-31 23:10:09,HD,H,2300.000
115593,2021-12-31 22:10:09,2021-12-31 23:10:09,HD,E,NaN


In [302]:
# DATA CLEANING PROCESS
# Isolate to just Type "E" which refers to Electricity
bingData = bingData[bingData['TYPE'] == 'E']

# Add Time Information to bingData Data Frame
bingData['time'] = pd.to_datetime(bingData['START_TIME'])
bingData['Time'] = pd.to_datetime(bingData['time']).dt.time
bingData['Date'] = pd.to_datetime(bingData['time']).dt.date
bingData['Week'] = bingData['time'].dt.strftime("%U")
bingData['Day'] = bingData['time'].dt.strftime("%j")
bingData['Day'] = pd.to_numeric(bingData['Day'], errors='coerce')
bingData['Weekday'] = bingData['time'].dt.strftime("%w")
bingData['Hour'] = pd.to_datetime(bingData['time']).dt.hour


C:\Users\aaliy\AppData\Local\Temp/ipykernel_13184/823083273.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\aaliy\AppData\Local\Temp/ipykernel_13184/823083273.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\aaliy\AppData\Local\Temp/ipykernel_13184/823083273.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

In [303]:
LHData = bingData[bingData['BUILDING_ID'] == 'LH']

#Found Max Demand for Each Day
idx = LHData.groupby(['Date'])['VALUE'].transform(max) == LHData['VALUE']
LHData = LHData[idx].drop_duplicates(['Date'], keep='first', ignore_index=True)
LHData

,START_TIME,END_TIME,BUILDING_ID,TYPE,VALUE,time,Time,Date,Week,Day,Weekday,Hour
0,2019-01-01 11:10:01,2019-01-01 12:10:01,LH,E,127.471,2019-01-01 11:10:01,11:10:01,2019-01-01,00,1,2,11
1,2019-01-02 13:10:04,2019-01-02 14:10:04,LH,E,133.110,2019-01-02 13:10:04,13:10:04,2019-01-02,00,2,3,13
2,2019-01-03 12:10:03,2019-01-03 13:10:03,LH,E,131.252,2019-01-03 12:10:03,12:10:03,2019-01-03,00,3,4,12
3,2019-01-04 12:10:10,2019-01-04 13:10:10,LH,E,130.482,2019-01-04 12:10:10,12:10:10,2019-01-04,00,4,5,12
4,2019-01-05 02:10:02,2019-01-05 03:10:02,LH,E,66.718,2019-01-05 02:10:02,02:10:02,2019-01-05,00,5,6,2
...,...,...,...,...,...,...,...,...,...,...,...,...
1013,2021-12-27 23:10:09,2021-12-28 00:10:09,LH,E,58.443,2021-12-27 23:10:09,23:10:09,2021-12-27,52,361,1,23
1014,2021-12-28 00:10:03,2021-12-28 01:10:03,LH,E,58.055,2021-12-28 00:10:03,00:10:03,2021-12-28,52,362,2,0
1015,2021-12-29 13:10:01,2021-12-29 14:10:01,LH,E,59.199,2021-12-29 13:10:01,13:10:01,2021-12-29,52,363,3,13
1016,2021-12-30 11:10:05,2021-12-30 12:10:05,LH,E,56.524,2021-12-30 11:10:05,11:10:05,2021-12-30,52,364,4,11


In [304]:
#Single the data to just the Fall Semester
#LHData['Week'] = pd.to_numeric(LHData['Week'], errors='coerce')
#LHData = LHData[LHData['Week'] >= 31]

In [305]:
px.line(y=LHData['VALUE'],x=LHData['Date'])

In [306]:
LHData.describe()
px.box(x=LHData['VALUE'])

In [307]:
px.scatter(x=LHData['Date'],y=LHData['VALUE'])


In [308]:
#Imported weather Data
weather = pd.read_csv('weatherData2019-2021.csv')
weather = pd.DataFrame({'DateTime':weather['DATE'], 'Temp':weather['HourlyDryBulbTemperature'],'Humidity':weather['HourlyRelativeHumidity'],'Wind Speed':weather['HourlyWindGustSpeed'],'SkyConditions':weather['HourlySkyConditions']}).dropna()
weather

weather['time'] = pd.to_datetime(weather['DateTime'])
weather['Time'] = pd.to_datetime(weather['time']).dt.time
weather['Date'] = pd.to_datetime(weather['time']).dt.date
weather['Week'] = weather['time'].dt.strftime("%U")
weather['Day'] = weather['time'].dt.strftime("%j")
weather['Day'] = pd.to_numeric(weather['Day'], errors='coerce')
weather['Weekday'] = weather['time'].dt.strftime("%w")
weather['Hour'] = pd.to_datetime(weather['time']).dt.hour

#Single to fall semester
weather['Week'] = pd.to_numeric(weather['Week'], errors='coerce')
weather = weather[weather['Week'] >= 31]

#Found Max Temp for Each Day
idx = weather.groupby(['Date'])['Temp'].transform(max) == weather['Temp']
weatherTemp = weather[idx].drop_duplicates(['Date'], keep='first', ignore_index=True)

C:\Users\aaliy\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning:

Columns (42,43,50,52,74,76,90) have mixed types.Specify dtype option on import or set low_memory=False.



In [309]:
#Found Max Humidity for Each Day
idx = weather.groupby(['Date'])['Humidity'].transform(max) == weather['Humidity']
weatherHumid = weather[idx].drop_duplicates(['Date'], keep='first', ignore_index=True)

In [310]:
#Found Max Wind Speed for Each Day
#WIND SPEED HAS MISSING DATA FOR THE GIVEN DATA PERIOD - another limitation
idx = weather.groupby(['Date'])['Humidity'].transform(max) == weather['Humidity']
weatherWind = weather[idx].drop_duplicates(['Date'], keep='first', ignore_index=True)

In [311]:
#Combine Max Temp + Max Humidity
weatherMax = pd.DataFrame({'Temp':weatherTemp['Temp'],'Humidity':weatherHumid['Humidity'],'Wind Speed':weatherWind['Wind Speed'],'Date':weatherTemp['Date'],'SkyConditions':weatherTemp['SkyConditions']}).dropna()
#MANUAL SOLUTION
#Remove November 8th 2020 to January 11th 2021 
weatherMax = weatherMax[(weatherMax.index < 248) | (weatherMax.index > 301)]
weatherMax

,Temp,Humidity,Wind Speed,Date,SkyConditions
0,81.0,94.0,22.0,2019-08-06,BKN:07 38 BKN:07 47 OVC:08 120
1,77.0,71.0,18.0,2019-08-07,SCT:04 29
2,78.0,78.0,47.0,2019-08-08,SCT:04 38 BKN:07 80
3,72.0,83.0,21.0,2019-08-09,FEW:02 55
4,70.0,73.0,18.0,2019-08-10,SCT:04 50 SCT:04 65 BKN:07 80
...,...,...,...,...,...
243,39.0,76.0,22.0,2021-11-16,FEW:02 50
244,48.0,80.0,21.0,2021-11-17,CLR:00
245,59.0,93.0,23.0,2021-11-18,FEW:02 120
246,38.0,92.0,17.0,2021-11-19,BKN:07 45 BKN:07 50


In [312]:
weatherMax['SkyConditions'] = weatherMax['SkyConditions'].str.extract(':(.{,2})') 
weatherMax['SkyConditions'] = pd.to_numeric(weatherMax['SkyConditions'],errors='coerce')
weatherMax

,Temp,Humidity,Wind Speed,Date,SkyConditions
0,81.0,94.0,22.0,2019-08-06,7.0
1,77.0,71.0,18.0,2019-08-07,4.0
2,78.0,78.0,47.0,2019-08-08,4.0
3,72.0,83.0,21.0,2019-08-09,2.0
4,70.0,73.0,18.0,2019-08-10,4.0
...,...,...,...,...,...
243,39.0,76.0,22.0,2021-11-16,2.0
244,48.0,80.0,21.0,2021-11-17,0.0
245,59.0,93.0,23.0,2021-11-18,2.0
246,38.0,92.0,17.0,2021-11-19,7.0


In [313]:
#DYNAMIC SOLUTION to remove the data from both data frames if missing data
#Merge johnson and weather data
mergedData = pd.merge(weatherMax, LHData, on=['Date'], how='inner')
mergedData['Weekday'] = pd.to_numeric(mergedData['Weekday'], errors='coerce')

mergedData['Wind Speed'].corr(mergedData['Wind Speed'])
fig = px.line(mergedData, y='VALUE',x='Date')
fig.layout = dict(title='Lecture Hall', xaxis = dict(type="category", categoryorder='category ascending'))
fig.show()

In [314]:
#HEAT INDEX
def heatIndex(T, RH):
    heatIndex = 0.5 * (T + 61.0 + ((T-68.0)*1.2) + (RH*0.094))
    heatIndex = (heatIndex + T)/2
    
    if heatIndex > 80:
        heatIndex = -42.379 + 2.04901523*T + 10.14333127*RH - .22475541*T*RH - .00683783*T*T - .05481717*RH*RH + .00122874*T*T*RH + .00085282*T*RH*RH - .00000199*T*T*RH*RH;
    
    if (RH < 13) and (80 < T < 112):
        adjustment = ((13-RH)/4)*np.sqrt((17-np.abs(T-95.))/17)
        heatIndex = heatIndex - adjustment
    
    if (RH > 85) and (80 < T < 87):
        adjustment = ((RH-85)/10) * ((87-T)/5)
        heatIndex = heatIndex + adjustment
        
    return heatIndex

mergedData['Heat Index'] = mergedData.apply(lambda x: heatIndex(x['Temp'], x['Humidity']), axis=1)
mergedData


,Temp,Humidity,Wind Speed,Date,SkyConditions,START_TIME,END_TIME,BUILDING_ID,TYPE,VALUE,time,Time,Week,Day,Weekday,Hour,Heat Index
0,81.0,94.0,22.0,2019-08-06,7.0,2019-08-06 14:10:01,2019-08-06 15:10:01,LH,E,216.926,2019-08-06 14:10:01,14:10:01,31,218,2,14,90.445557
1,77.0,71.0,18.0,2019-08-07,4.0,2019-08-07 12:10:08,2019-08-07 13:10:08,LH,E,220.631,2019-08-07 12:10:08,12:10:08,31,219,3,12,77.368500
2,78.0,78.0,47.0,2019-08-08,4.0,2019-08-08 15:10:06,2019-08-08 16:10:06,LH,E,217.943,2019-08-08 15:10:06,15:10:06,31,220,4,15,78.583000
3,72.0,83.0,21.0,2019-08-09,2.0,2019-08-09 12:10:05,2019-08-09 13:10:05,LH,E,178.474,2019-08-09 12:10:05,12:10:05,31,221,5,12,72.400500
4,70.0,73.0,18.0,2019-08-10,4.0,2019-08-10 18:10:05,2019-08-10 19:10:05,LH,E,114.039,2019-08-10 18:10:05,18:10:05,31,222,6,18,70.065500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,39.0,76.0,22.0,2021-11-16,2.0,2021-11-16 17:10:10,2021-11-16 18:10:10,LH,E,186.039,2021-11-16 17:10:10,17:10:10,46,320,2,17,37.586000
204,48.0,80.0,21.0,2021-11-17,0.0,2021-11-17 15:10:03,2021-11-17 16:10:03,LH,E,186.257,2021-11-17 15:10:03,15:10:03,46,321,3,15,47.130000
205,59.0,93.0,23.0,2021-11-18,2.0,2021-11-18 15:10:09,2021-11-18 16:10:09,LH,E,195.629,2021-11-18 15:10:09,15:10:09,46,322,4,15,58.985500
206,38.0,92.0,17.0,2021-11-19,7.0,2021-11-19 15:10:08,2021-11-19 16:10:08,LH,E,178.093,2021-11-19 15:10:08,15:10:08,46,323,5,15,36.912000


In [315]:
#US BANK HOLIDAYS
holidays = pd.read_csv('US BANK holidays.csv')
holidays = holidays.drop(columns=['Unnamed: 0']);

mergedData['Date'] = pd.to_datetime(mergedData.Date, format='%Y-%m-%d')
mergedData['Date'] = mergedData['Date'].dt.strftime('%Y-%m-%d')


def holiday(data):

    global add

    add=0
    for x in holidays['date']:

      if (data == holidays.at[add,'date']):
        weekendIndex = 0
        add=0
     #   print("STOP")
        return weekendIndex
      else:
        weekendIndex = 1
        add = add+1
       # print("DROP")

   # print("ROLL")
    add=0
    return weekendIndex

#mergedData['Weekday Index'] = mergedData.apply(lambda x: holiday(x['Date']),axis=1)
mergedData

,Temp,Humidity,Wind Speed,Date,SkyConditions,START_TIME,END_TIME,BUILDING_ID,TYPE,VALUE,time,Time,Week,Day,Weekday,Hour,Heat Index
0,81.0,94.0,22.0,2019-08-06,7.0,2019-08-06 14:10:01,2019-08-06 15:10:01,LH,E,216.926,2019-08-06 14:10:01,14:10:01,31,218,2,14,90.445557
1,77.0,71.0,18.0,2019-08-07,4.0,2019-08-07 12:10:08,2019-08-07 13:10:08,LH,E,220.631,2019-08-07 12:10:08,12:10:08,31,219,3,12,77.368500
2,78.0,78.0,47.0,2019-08-08,4.0,2019-08-08 15:10:06,2019-08-08 16:10:06,LH,E,217.943,2019-08-08 15:10:06,15:10:06,31,220,4,15,78.583000
3,72.0,83.0,21.0,2019-08-09,2.0,2019-08-09 12:10:05,2019-08-09 13:10:05,LH,E,178.474,2019-08-09 12:10:05,12:10:05,31,221,5,12,72.400500
4,70.0,73.0,18.0,2019-08-10,4.0,2019-08-10 18:10:05,2019-08-10 19:10:05,LH,E,114.039,2019-08-10 18:10:05,18:10:05,31,222,6,18,70.065500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,39.0,76.0,22.0,2021-11-16,2.0,2021-11-16 17:10:10,2021-11-16 18:10:10,LH,E,186.039,2021-11-16 17:10:10,17:10:10,46,320,2,17,37.586000
204,48.0,80.0,21.0,2021-11-17,0.0,2021-11-17 15:10:03,2021-11-17 16:10:03,LH,E,186.257,2021-11-17 15:10:03,15:10:03,46,321,3,15,47.130000
205,59.0,93.0,23.0,2021-11-18,2.0,2021-11-18 15:10:09,2021-11-18 16:10:09,LH,E,195.629,2021-11-18 15:10:09,15:10:09,46,322,4,15,58.985500
206,38.0,92.0,17.0,2021-11-19,7.0,2021-11-19 15:10:08,2021-11-19 16:10:08,LH,E,178.093,2021-11-19 15:10:08,15:10:08,46,323,5,15,36.912000


In [316]:
#Create weekday index
#0 for Weekdays being 5 or 6 (weekends) OR if it's a holiday
#1 for anything else
def createWeekdayIndex(weekday, date):
    global add
    add = 0


    for x in holidays['date']:
      if (date == holidays.at[add,'date']):
        weekdayIndex = 0
        add=0
        return weekdayIndex
      else:
        weekdayIndex = 1
        add = add+1
    add=0

    if(weekday == 5) or (weekday == 6):
        weekdayIndex = 0
    return weekdayIndex

mergedData['Weekday Index'] = mergedData.apply(lambda x: createWeekdayIndex(x['Weekday'],x['Date']),axis=1)

#QUICK SPEDEDY TEST
#mergedData = mergedData[mergedData['VALUE'] < 312]
mergedData

,Temp,Humidity,Wind Speed,Date,SkyConditions,START_TIME,END_TIME,BUILDING_ID,TYPE,VALUE,time,Time,Week,Day,Weekday,Hour,Heat Index,Weekday Index
0,81.0,94.0,22.0,2019-08-06,7.0,2019-08-06 14:10:01,2019-08-06 15:10:01,LH,E,216.926,2019-08-06 14:10:01,14:10:01,31,218,2,14,90.445557,1
1,77.0,71.0,18.0,2019-08-07,4.0,2019-08-07 12:10:08,2019-08-07 13:10:08,LH,E,220.631,2019-08-07 12:10:08,12:10:08,31,219,3,12,77.368500,1
2,78.0,78.0,47.0,2019-08-08,4.0,2019-08-08 15:10:06,2019-08-08 16:10:06,LH,E,217.943,2019-08-08 15:10:06,15:10:06,31,220,4,15,78.583000,1
3,72.0,83.0,21.0,2019-08-09,2.0,2019-08-09 12:10:05,2019-08-09 13:10:05,LH,E,178.474,2019-08-09 12:10:05,12:10:05,31,221,5,12,72.400500,0
4,70.0,73.0,18.0,2019-08-10,4.0,2019-08-10 18:10:05,2019-08-10 19:10:05,LH,E,114.039,2019-08-10 18:10:05,18:10:05,31,222,6,18,70.065500,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,39.0,76.0,22.0,2021-11-16,2.0,2021-11-16 17:10:10,2021-11-16 18:10:10,LH,E,186.039,2021-11-16 17:10:10,17:10:10,46,320,2,17,37.586000,1
204,48.0,80.0,21.0,2021-11-17,0.0,2021-11-17 15:10:03,2021-11-17 16:10:03,LH,E,186.257,2021-11-17 15:10:03,15:10:03,46,321,3,15,47.130000,1
205,59.0,93.0,23.0,2021-11-18,2.0,2021-11-18 15:10:09,2021-11-18 16:10:09,LH,E,195.629,2021-11-18 15:10:09,15:10:09,46,322,4,15,58.985500,1
206,38.0,92.0,17.0,2021-11-19,7.0,2021-11-19 15:10:08,2021-11-19 16:10:08,LH,E,178.093,2021-11-19 15:10:08,15:10:08,46,323,5,15,36.912000,0


In [317]:
from sklearn.linear_model import LinearRegression
from sklearn import metrics
data = pd.DataFrame({'Electricity':mergedData['VALUE'],'Weekday Index':mergedData['Weekday Index'], 'Temp':mergedData['Temp'],'Humidity':mergedData['Humidity'],'Wind Speed':mergedData['Wind Speed'],'Heat Index':mergedData['Heat Index'],'Cloud Index':mergedData['SkyConditions']}).dropna()
std_scaler = StandardScaler()
std_scaler

features = pd.DataFrame(std_scaler.fit_transform(features), columns=features.columns)

x_train = np.asarray([data['Weekday Index'],data['Temp'], data['Humidity'], data['Wind Speed'],data['Heat Index'],data['Cloud Index']]).T
y_train = np.asarray(data['Electricity'])



In [318]:
model = LinearRegression()
#calculate beta using train
model.fit(x_train,y_train)
y_pred = model.predict(x_train)

model.coef_


array([30.16563294,  3.48777337,  0.64210554, -1.51304209, -1.38588092,
       -0.31620812])

In [319]:
model.score(x_train,y_train) #This is the R^2 value


0.35034944122644907

In [320]:
data.corr()

,Electricity,Weekday Index,Temp,Humidity,Wind Speed,Heat Index,Cloud Index
Electricity,1.000000,0.280809,0.518730,0.057728,-0.105736,0.515197,-0.096834
Weekday Index,0.280809,1.000000,0.100754,0.053594,0.023896,0.103557,-0.013172
Temp,0.518730,0.100754,1.000000,-0.165421,-0.081003,0.993073,-0.277543
Humidity,0.057728,0.053594,-0.165421,1.000000,0.205437,-0.126866,0.448369
Wind Speed,-0.105736,0.023896,-0.081003,0.205437,1.000000,-0.065322,0.075224
Heat Index,0.515197,0.103557,0.993073,-0.126866,-0.065322,1.000000,-0.258827
Cloud Index,-0.096834,-0.013172,-0.277543,0.448369,0.075224,-0.258827,1.000000


In [321]:
px.box(x=y_pred)

In [322]:
px.box(x=data['Electricity'])

In [323]:
data = data[data['Electricity'] < 250]
data = data[data['Electricity'] > 70]

px.box(x=data['Electricity'])


In [324]:
data

,Electricity,Weekday Index,Temp,Humidity,Wind Speed,Heat Index,Cloud Index
0,216.926,1,81.0,94.0,22.0,90.445557,7.0
1,220.631,1,77.0,71.0,18.0,77.368500,4.0
2,217.943,1,78.0,78.0,47.0,78.583000,4.0
3,178.474,0,72.0,83.0,21.0,72.400500,2.0
4,114.039,0,70.0,73.0,18.0,70.065500,4.0
...,...,...,...,...,...,...,...
203,186.039,1,39.0,76.0,22.0,37.586000,2.0
204,186.257,1,48.0,80.0,21.0,47.130000,0.0
205,195.629,1,59.0,93.0,23.0,58.985500,2.0
206,178.093,0,38.0,92.0,17.0,36.912000,7.0


In [325]:
x_train = np.asarray([data['Weekday Index'],data['Temp'], data['Humidity'], data['Wind Speed'],data['Heat Index'],data['Cloud Index']]).T
y_train = np.asarray(data['Electricity'])

model = LinearRegression()
#calculate beta using train
model.fit(x_train,y_train)
y_pred = model.predict(x_train)

model.coef_
model.score(x_train,y_train)

0.23218755365996824

In [326]:
px.box(y_pred)

In [327]:
px.box(data['Electricity'])